In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
df = pd.read_excel('AJGT.xlsx',header = 0)
df = df[['Feed','Sentiment']]
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
df.columns= [DATA_COLUMN,LABEL_COLUMN]


In [5]:
data_train, data_test = train_test_split(df,test_size=0.2,random_state=42)

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers.data.processors.utils import InputFeatures
from transformers import AutoTokenizer


class SADataset(Dataset):
  def __init__(self, texts, labels, model_name, max_len, label_map):
   
    self.texts = texts
    self.labels = labels
    self.label_map = label_map
    self.tokenizer_name = model_name
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.max_len = max_len

  def __len__(self):
   
    return len(self.texts)

  def __getitem__(self, item):
    
    text = str(self.texts[item])
    label = self.labels[item]

    input_dict = self.tokenizer(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation= True
      )

    return InputFeatures(input_ids=input_dict["input_ids"],
                         token_type_ids=input_dict['token_type_ids'],
                         attention_mask=input_dict["attention_mask"],
                         label=self.label_map[self.labels[item]])

c:\Users\Khaled\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    assert len(preds) == len(pred.label_ids)

   
    acc = accuracy_score(pred.label_ids, preds)
    macro_f1 = f1_score(pred.label_ids, preds, average='macro')
    macro_f1_pos_neg = f1_score(pred.label_ids, preds, labels=[0, 1], average='macro') # Adjust labels as needed
    macro_precision = precision_score(pred.label_ids, preds, average='macro')
    macro_recall = recall_score(pred.label_ids, preds, average='macro')

    return {
        'macro_f1': macro_f1,
        'macro_f1_pos_neg': macro_f1_pos_neg,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'accuracy': acc
    }

In [8]:
from arabert.preprocess import ArabertPreprocessor

model_name = 'aubmindlab/bert-base-arabertv02'
arabert_prep = ArabertPreprocessor(model_name)

In [9]:
data_train[DATA_COLUMN] = data_train[DATA_COLUMN].apply(lambda x: arabert_prep.preprocess(x))
data_test[DATA_COLUMN] = data_test[DATA_COLUMN].apply(lambda x: arabert_prep.preprocess(x))

In [10]:
label_list = list(data_test[LABEL_COLUMN].unique())
label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

{'Negative': 0, 'Positive': 1}


In [11]:
max_len = 256
train_dataset = SADataset(
   texts = data_train[DATA_COLUMN].to_list(),
   labels = data_train[LABEL_COLUMN].to_list(),
   model_name = model_name,
   max_len = 256,
   label_map = label_map
)

test_dataset = SADataset(
   texts = data_test[DATA_COLUMN].to_list(),
   labels = data_test[LABEL_COLUMN].to_list(),
   model_name = model_name,
   max_len = 256,
   label_map = label_map
    )

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer, TrainingArguments


In [14]:

training_args = TrainingArguments(
    output_dir="./train",
    adam_epsilon=1e-8,
    learning_rate=5e-5,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=8,
    do_eval=True,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_macro_f1',
    greater_is_better=True,
    seed=42
)


In [15]:
training_args.__dict__

{'output_dir': './train',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': True,
 'do_predict': False,
 'eval_strategy': <IntervalStrategy.EPOCH: 'epoch'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 16,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 2,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'torch_empty_cache_steps': None,
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 8,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,
 'lr_scheduler_kwargs': {},
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './train\\runs\\Apr21_02-15-09_DESKTOP-AP1M4LC',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': Fal

In [16]:
trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = train_dataset,
   eval_dataset = test_dataset,
   compute_metrics = compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy
1,No log,0.175410,0.938402,0.938402,0.939979,0.937358,0.938889
2,No log,0.169439,0.944334,0.944334,0.943966,0.946170,0.944444
3,No log,0.240042,0.949777,0.949777,0.949472,0.950141,0.950000


KeyboardInterrupt: 

#  Saving the best model

In [17]:
trainer.model.config.label2id = label_map
inv_label_map = { v:k for k, v in label_map.items()}
trainer.model.config.id2label = inv_label_map

In [18]:
#save the model in the folder
trainer.save_model("best_sa_model")
test_dataset.tokenizer.save_pretrained("best_sa_model")

('best_sa_model\\tokenizer_config.json',
 'best_sa_model\\special_tokens_map.json',
 'best_sa_model\\vocab.txt',
 'best_sa_model\\added_tokens.json',
 'best_sa_model\\tokenizer.json')

In [ ]:
from transformers import pipeline
pipe = pipeline(
        "sentiment-analysis",
        model = "best_sa_model",
        device=0, 
        )

Device set to use cpu


In [20]:
pipe("انا لا احبك")

[{'label': 'Negative', 'score': 0.9978159666061401}]

In [21]:
pipe("انا احبك")

[{'label': 'Positive', 'score': 0.9898937344551086}]

In [22]:
pipe("  الاكل ما عجبني للاسف")

[{'label': 'Negative', 'score': 0.9939164519309998}]

In [23]:
pipe(" جميل")

[{'label': 'Positive', 'score': 0.9953432083129883}]

In [24]:
pipe(" الخدمة كانت كب شينة لكن موخرا صارت سيئة")

[{'label': 'Negative', 'score': 0.9992189407348633}]

In [25]:
pipe("خدمة العملاء أكثر من مرة اتصل عليهم ولا يحلون لي مشكلتي")

[{'label': 'Negative', 'score': 0.9975154399871826}]

In [26]:
pipe("دكتور سلطان يده خفيفة جدا ما شاء الله")

[{'label': 'Positive', 'score': 0.985045850276947}]

In [27]:
pipe("الدكتور درسني في الجامعة فمة في الخلق والتواضع ")

[{'label': 'Positive', 'score': 0.9934831261634827}]

In [ ]:
# import shutil
# from google.colab import files

# # Compress the model directory into a zip file
# shutil.make_archive('best_sa_model', 'zip', 'best_sa_model')

# # Download the zip file
# files.download('best_sa_model.zip')


In [ ]:
# import pickle

# # save the iris classification model as a pickle file
# model_pkl_file = "AHSP_v0"

# with open(model_pkl_file, 'wb') as file:
#     pickle.dump(model, file)


In [ ]:
# import joblib

# # save model with joblib
# filename = 'joblib_model.sav'
# joblib.dump(model, filename)